# Cross-Sections

This section demonstrates how to extract cross-sections from an unstructured grid using UXarray, which allows the analysis and visualization across slices of grids.


In [ ]:
import uxarray as ux
import geoviews.feature as gf

import cartopy.crs as ccrs
import geoviews as gv

projection = ccrs.Robinson()

## Data

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)
uxds["psi"].plot(
    cmap="inferno",
    periodic_elements="split",
    projection=projection,
    title="Global Plot",
)

## Constant Latitude

Cross-sections along constant latitude lines can be obtained using the ``.cross_section.constant_latitude`` method, available for both ``ux.Grid`` and ``ux.DataArray`` objects. This functionality allows users to extract and analyze slices of data at specified latitudes, providing insights into variations along horizontal sections of the grid.


For example, we can obtain a cross-section at 30 degrees latitude by doing the following:

In [ ]:
lat = 30

uxda_constant_lat = uxds["psi"].cross_section.constant_latitude(lat)

Since the result is a new ``UxDataArray``, we can directly plot the result to see the cross-section.

In [ ]:
(
    uxda_constant_lat.plot(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross Section at {lat} degrees latitude",
    )
    * gf.grid(projection=projection)
)

You can also perform operations on the cross-section, such as taking the mean.

In [ ]:
print(f"Global Mean: {uxds['psi'].data.mean()}")
print(f"Mean at {lat} degrees lat: {uxda_constant_lat.data.mean()}")

## Constant Longitude

```{warning}
Constant longitude cross sections are not yet supported.
```

## Arbitrary Great Circle Arc (GCA)

```{warning}
Arbitrary great circle arc cross sections are not yet supported.
```